In [ ]:
consultas = ["INSERT INTO users(name, age, alias) VALUES ('Marcela Sena', 29, 'MarceStarlet')",
              'DELETE FROM nombre_tabla WHERE nombre_columna = valor',
             'SELECT * FROM CLIENTES',
             'SELECT idCliente, nombre, descripcion FROM CLIENTES',
             "SELECT cp, ciudad FROM DIRECCION","INSERT INTO alumnos VALUES('07680654','Carlos','Zarate', 23,78.0)"]
consultas.append("DELETE FROM personas WHERE nombre = 'LUIS'")
           

# **ANALIZADOR LEXICO**

In [ ]:
tokens = [
    #SIMBOLOS ESPECIALES
    ('PARIZQ','('),
    ('PARDER',')'),
    ('COMA',','),
    ('PUNTO','.'),
    ('PTOCOMA',';'),
    ('COMILLA',"'"),
    #OPERADORES
    ('MENORQUE','<'),
    ('MAYORQUE','>'),
    ('MENORIGUAL','<='),
    ('MAYORIGUAL','>='),
    ('IGUAL','='),
    ('DISTINTO','!='),
    ('Y','AND'),
    ('O','OR'),
    ('POR','*'),
    #PALABRA RESERVADA
    ('CREATE','CREATE'),
    ('SELECT','SELECT'),
    ('DISTINCT','DISTINCT'),
    ('INSERT','INSERT'),
    ('UPDATE','UPDATE'),
    ('DELETE','DELETE'),
    ('FROM','FROM'),
    ('WHERE','WHERE'),
    ('GROUP','GROUP'),
    ('BY','BY'),
    ('INTO','INTO'),
    ('VALUES','VALUES'),
    ('JOIN','JOIN')
]
def is_operador(s):
  if s == '(' or s == ')' or s == ',' or s == '.' or s == "'" or s == '<' or s == '>' or s == '=' or s == '*' or s == '!' or s == ';':
    return True
  return False
def analisis_lexico(consulta):
  lexema = []
  datos = []
  dato = ""
  operador = ""
  consulta += " "
  #Separo los datos de mi consulta en Arrays
  for i in consulta:
    if is_operador(i):
      if len(dato) > 0:
        datos.append(dato)
        dato = "";
      if i != '>' and i != '<' and i != '!':
        if operador == '>' or operador == '<' or operador == '!':
          operador += i
          datos.append(operador)
          operador = ""
        else:
          datos.append(i)
      else:
        operador += i
    elif i != ' ':
      if len(operador) > 0:
        datos.append(operador)
        operador = "";
      dato += i;
    else:
      if len(operador) > 0:
        datos.append(operador)
        operador = "";
      if len(dato) > 0:
        datos.append(dato)
        dato = "";
  #Procedo a identificar, cada dato separado 'anteriormente asd 21234 234b wda ', con los tokens ya declarados 
  i = 0
  decimal = 0
  string = 0
  for dato_valor in datos: #datos[i]
    is_token = False
    for name, valor in tokens:
      if decimal > 0:
        decimal-=1
        is_token = True
        break
      if string > 0:
        string-=1
        is_token = True
        break
      if valor == dato_valor:
        if valor != "'":
          lexema.append((name,dato_valor))
          is_token = True
        else: # CREAR EL STRING 
          cadena = "'"
          string += 1
          while datos[i+string] != "'":
            cadena += datos[i+string]
            string += 1
            if datos[i+string] == "'":
              cadena += "'"
            else:
              cadena += ' '
          lexema.append(('STRING',cadena))
          is_token = True
        break 
    if is_token == False:
      if dato_valor.isdigit():              #  i   i+1  i+2
        #Identificamos si hay decimales 19.2 [19]  [.]  [2]
        if datos[i+1] == '.' and datos[i+2].isdigit():
          lexema.append(('DECIMAL',datos[i]+datos[i+1]+datos[i+2]))
          decimal += 2
        else:
          lexema.append(('NUMERO',dato_valor))
      elif dato_valor[0].isalpha():
        lexema.append(('ID',dato_valor))
      else:
        lexema.append(('NO VALIDO',dato_valor))
    i+=1
  return lexema
consulta = "INSERT INTO alumnos VALUES('07680654','Carlos','Zarate', 23,78.0)"
#Analisis Lexico
lexema = analisis_lexico(consulta)
index = 0
for name, valor in lexema:
  print('[',index,']',name,'\t  ==>  ',valor)
  index+=1

[ 0 ] INSERT 	  ==>   INSERT
[ 1 ] INTO 	  ==>   INTO
[ 2 ] ID 	  ==>   alumnos
[ 3 ] VALUES 	  ==>   VALUES
[ 4 ] PARIZQ 	  ==>   (
[ 5 ] STRING 	  ==>   '07680654'
[ 6 ] COMA 	  ==>   ,
[ 7 ] STRING 	  ==>   'Carlos'
[ 8 ] COMA 	  ==>   ,
[ 9 ] STRING 	  ==>   'Zarate'
[ 10 ] COMA 	  ==>   ,
[ 11 ] NUMERO 	  ==>   23
[ 12 ] COMA 	  ==>   ,
[ 13 ] DECIMAL 	  ==>   78.0
[ 14 ] PARDER 	  ==>   )


# **ANALIZADOR SINTACTICO**

In [ ]:
class Nodo:
    def __init__(self, dato):
        self.dato = dato
        self.hijo = []

class Arbol:
    def __init__(self, dato):
        self.raiz = Nodo(dato)

    def __agregar_recursivo(self, nodo, dato,i):
        existe = False
        n = 0
        for name in nodo.hijo:
            if dato[i] == name.dato:
                existe = True
                break
            n += 1
        if existe == False:
            nodo.hijo.append(Nodo(dato[i]))
            i+=1
            if i < len(dato):
              n = len(nodo.hijo) - 1
              self.__agregar_recursivo(nodo.hijo[n],dato,i)
        else:
            i+=1
            if i < len(dato):
              self.__agregar_recursivo(nodo.hijo[n],dato,i)

    def agregar(self, dato):
        self.__agregar_recursivo(self.raiz, dato,0)

    def __imprimir(self,nodo,n):
        i = 0
        string = ""
        for name in nodo.hijo:
            string += name.dato + "-" 
            self.__imprimir(nodo.hijo[i],n+1)
            i += 1
        print(string,n)

    def imprimir(self):
        self.__imprimir(self.raiz,1)

    def __analizar(self,nodo,lexema,n):
        i = 0
        existe = False
        for name in nodo.hijo:
            #print(name.dato, lexema[n][0], n)
            if name.dato == lexema[n][0]:
                existe = True
                return self.__analizar(nodo.hijo[i],lexema,n+1)
                #break
            i += 1
        if existe == False and n < len(lexema):
            print('ERROR DE SINTAXIS: en la linea',n,'en "',lexema[n][1],'"')
            return True
        return False

    def analizar(self, lexema):
        return self.__analizar(self.raiz,lexema,0)

Sintactico = Arbol('PROGRAMA')
for dato in consultas:
    lexema = analisis_lexico(dato)
    nuevo_token = []
    for name,valor in lexema:
        nuevo_token.append(name)
    Sintactico.agregar(nuevo_token)

consulta = "INSERT INTO alumnos VALUES('07680654'*'Carlos','Zarate', 23,78.0)"
lexema2 = analisis_lexico(consulta)
Sintactico.analizar(lexema2)


ERROR DE SINTAXIS: en la linea 6 en " * "


True

In [ ]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where, Function
stringa1 = 'SELECT * FROM people'         #//////////////////////////
#stringa2 = 'SELECT id, user_id, status FROM people order by status asc limit 3'  #//////////////////////////
stringa3 = 'SELECT user_id FROM people'  #//////////////////////////

def select(tokens):
  table = "" #aqui entran en toda la cadena 
  where_found = False
  select_found = False
  limit_found = False
  wildcard_found = False
  wildcard_count_found = False
  distinct_found = False
  count_found = False
  output_parenthesis_select_attributes = ""
  order_by_found = False
  order_by_rule = ""
  count_field = ""
  count_field_found = False

# itera a través de todos los tokens para ubicar el nombre de la tabla y encontrar la condición where,
# que se almacena en el vector analizado.
# (es) parsed -> ['status', '=', '"D"', 'OR', 'name', '<=', '"Carlo"', 'AND', 'name', '!=', '"Saretta"']
  for token in tokens:
    if token.value == "SELECT":
      select_found = True
    if token.value == "*":
      wildcard_found = True
      select_found = False




    if isinstance(token, Identifier) and not select_found and not order_by_found:
      table = token.value
    if isinstance(token,Identifier) and select_found :
      select_found = False
      output_parenthesis_select_attributes = convert_single_select_attribute(token) 
    if isinstance(token,IdentifierList) and select_found:
      select_found = False
      output_parenthesis_select_attributes = convert_multiple_select_attributes(token)



    if isinstance(token, Where):
      where_found = True
      output = convert_where_condition(token)
      # Si los operadores lógicos estuvieran presentes en la condición where
      # luego utilícelos para la construcción de la consulta final, de lo contrario
      # si se trataba de una condición simple, cree la consulta final con solo
      # el único selector presente.
      comma = ""

      if isinstance(output[0],LogicOperator):
        if distinct_found:
          output_parenthesis_distinct = '.distinct("' + distinct_value + '", ' + output[-1].created_string + ")"
          final_query = "db." + table + output_parenthesis_distinct
        if output_parenthesis_select_attributes != "" and not distinct_found:
          comma = ","
        if limit_found:
          limit_found = False
          final_query = "db."+ table +".find(" + output[-1].created_string + comma + " " + output_parenthesis_select_attributes + ")" + output_parenthesis_limit
        elif count_field_found:
          final_query = "db."+ table +".find(" + output[-1].created_string + comma + " " + output_parenthesis_select_attributes + "{" + count_field + ": {exists:true}}" + ")"
        else:
          final_query = "db."+ table +".find(" + output[-1].created_string + comma + " " + output_parenthesis_select_attributes + ")"

      elif output[1] == "like":
        original = output[2].replace('"', '')
        positions = [pos for pos, char in enumerate(original) if char == "%"]
        like_arg = original.replace("%", "/")
        if len(positions) == 1:
          if positions[0] == 0:
             like_arg = like_arg[:len(original)] + "$/" + like_arg[len(original):]
          elif positions[0] == len(original) - 1 :
            like_arg = like_arg[:0] + "/^" + like_arg[0:]
        final_query = "db." + table + ".find({" + output[0] + ": " + like_arg + "})"



  if not where_found:
    if distinct_found:
      final_query = "db."+ str(table) + output_parenthesis_distinct
    else:
      if wildcard_found:
        final_query = "db."+str(table)+".find({})"
      elif count_field_found:
        final_query = "db."+ table +".find({" + count_field + ": {exists:true}}" + ")" 
      elif output_parenthesis_select_attributes != "" :
        final_query = "db."+str(table)+".find({}," + output_parenthesis_select_attributes + ")"
      else:
        final_query = "db."+str(table)+".find({})"
  if order_by_found:
    final_query = final_query + ".sort({ " + order_by_column + ": " + order_by_rule + "})"
  if limit_found:
    final_query = final_query + output_parenthesis_limit
  if count_found:
    final_query = final_query + ".count()"

  return(final_query)






# Los operadores se traducen a mongoDB,
# comenzando por aquellos con mayor prioridad y almacenando el resultado
# parcial en el atributo created_string del bloque. Los operadores posteriores
# que tienen un mapeo izquierdo o derecho que el bloque recién ejecutado se construirá
# incrementalmente el resultado a partir del valor de created_string.
# El resultado final de la traducción estará contenido en el atributo


# Clase del operador lógico Y / O. La pos indica la posición dentro de la condición where,
# (y por lo tanto actúa como id), el ttype indica el tipo Y / O, la prioridad el orden de ejecución, izquierda y derecha
# las subcondiciones izquierda y derecha, mientras que el resultado se almacena en created_string
# de su traducción a la sintaxis mongoDB.
class LogicOperator:
    def __init__(self, pos = None, ttype = None, priority = None, left = None, right = None, created_string = None):
      self.pos = pos
      self.ttype = ttype
      self.priority = priority
      self.left = left
      self.right = right
      self.created_string = created_string
    def __str__(self):
      return (str(self.__class__) + ": " + str(self.__dict__))


# La clase de bloque indica una subcondición que precede o sigue a un operador lógico
# en la condición inicial where. Por lo tanto, se caracteriza por un id (posición del bloque),
# da una traducción en sql (sql_value) y otra en mognodb (mongo_value).
# El atributo mapped_by se utiliza para mapear un operador posterior a uno anterior.
# que ya ha asignado ese bloque.
class Block:
  def __init__(self, id, sql_value = None, mongo_value = None, mapped_by = None):
    self.id = id
    self.sql_value = sql_value
    self.mongo_value = mongo_value
    self.mapped_by = mapped_by
  def __str__(self):
    return (str(self.__class__) + ": " + str(self.__dict__))

# Función para encontrar un operador lógico en logic_ops por su id / pos
def find(list, id):
  result = None
  for item in list:
    if item.pos == id:
      result = item
  return result

def convert_single_select_attribute(token):
  if (token.value == "id"):
    output_parenthesis_select = "{_id: 1}"
  else:
    output_parenthesis_select = "{ " + token.value + ": 1, " + "_id: 0}"
  return output_parenthesis_select

def convert_multiple_select_attributes(token):
  elements = []
  id_found = False
  output_parenthesis_select = ""
  for elem in token:
    if isinstance(elem, Identifier):
      if (elem.value == "id"):
        id_found = True
      else:
        output = elem.value + ": 1"
        elements.append(output)
  
  output_string_noquotes = output_string.replace("'", "")
 
  if not id_found:
    output_parenthesis_select = "{ " + output_string_noquotes +  ", _id: 0 }"
  else: 
    output_parenthesis_select = "{ " + output_string_noquotes +  " }"

  return output_parenthesis_select
"""
Created on Tue Feb 16 12:55:55 2021

@author: JOEL
"""

import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where

# BLOCCO MAIN
stringa1 = 'DELETE FROM molise;'
stringa111 = 'DELETE FROM estudiates'
stringa2 = 'DELETE FROM people WHERE status = "D"'
#stringa3 = 'DELETE FROM people WHERE status = "D" and name <= "Carlo" or name != "Saretta"'
#stringa4 = 'DELETE FROM people WHERE status = "D" or name <= "Carlo" and name != "Saretta"'
#stringa5 = 'DELETE FROM people WHERE status = "D" and name <= "Carlo" and name != "Saretta"'
#stringa6 = 'DELETE FROM people WHERE status = "D" or name <= "Carlo" and name != "Saretta" or age >= 18'


# DELETE WHERE - elimina le tuple della tabella con una certa condizione
def delete(tokens):
  table = ""
  where_found = False
# scorre tutti i token per individuare il nome della tabella e trovare la condizione del where,
# che viene memorizzata nel vettore parsed.
# (es) parsed -> ['status', '=', '"D"', 'OR', 'name', '<=', '"Carlo"', 'AND', 'name', '!=', '"Saretta"']
  for token in tokens:
    if isinstance(token, Identifier):
      table = token.value
    if isinstance(token, Where):
      where_found = True
      output = convert_where_condition(token)
      # Se nella condizione del where erano presenti degli operatori logici 
      # allora utilizzali per la costruzione della query finale, altrimenti
      # se era una condizione semplice costruisci la query finale con solamente
      # l'unico selettore presente.
      if isinstance(output[0],LogicOperator):
        final_query = "db."+ table +".deleteMany(" + output[-1].created_string + ")"
      else:
        output_parenthesis = convert_condition_in_mongo(output) 
        final_query = "db." + table + ".deleteMany(" + str(output_parenthesis[0]) + ")"
  if not where_found:
    final_query = "db."+str(table)+".deleteMany({})"
  return(final_query)



# Una volta trovata la condizione del where, scorre tutti i suoi token 
# per memorizzare nel dizionario logic_op_positions il tipo di operatore logico 
# e la sua posizione all'interno della condizione where.
# (es) logic_op_positions -> {3: 'OR', 7: 'AND'}

def create_op_positions(parsed):
  logic_op_positions = {}
  for i, item in enumerate(parsed, start = 0):
    if item == "AND" or item == "OR":
      logic_op_positions[i] = "{0}".format(item)
  return logic_op_positions

# Crea il vettore 2D where_list_2D, che contiene al suo interno tante liste
# di token quante sono le sottocondizioni di cui è composto il where.
# (es) where_list_2D -> [['status', '=', '"D"'], ['name', '<=', '"Carlo"'], ['name', '!=', '"Saretta"']]

def create_subconditions_list(logic_op_positions, parsed):
  starting_pos = 0
  where_list_2D = []
  for key, value in logic_op_positions.items():
    temp_list = parsed[starting_pos:key]
    where_list_2D.append(temp_list)
    starting_pos = key + 1
    # Nel momento in cui viene raggiunto l'ultimo operatore logico, 
    # deve essere costruita l'ultima sottolista di where_list_2D prendendo 
    # tutti i token rimamente in parsed.
    if key == list(logic_op_positions.items())[-1][0]: 
      temp_list = parsed[starting_pos:len(parsed)]
      where_list_2D.append(temp_list)
  return where_list_2D

# A questo punto avviene la traduzione delle sottocondizione da sql a sintassi mongodb,
# inanzitutto traducendo il selettore (che è sempre l'elemento centrale di ogni sottolista)
# e poi memorizzando il risultato finale nel vettore output_parenthesis.
# (es) output_parenthesis -> ['{ status: { $eq: "D" }}', '{ name: { $lte: "Carlo" }}', '{ name: { $ne: "Saretta" }}']
def convert_subconditions_in_mongo(where_list_2D):
  output_parenthesis = []
  for item in where_list_2D:
    if item[1] == "=":
      selector = "$eq"
    elif item[1] == "!=":
      selector = "$ne"
    elif item[1] == ">":
      selector = "$gt"
    elif item[1] == ">=":
      selector = "$gte"
    elif item[1] == "<":
      selector = "$lt"
    elif item[1] == "<=":
      selector = "$lte"
    output_inner = "{ "+ item[0] +": { "+selector+": " + item[2] + " }"
    output_parenthesis.append(output_inner + "}")
  return output_parenthesis

# Viene creato il vettore logic_op_priorities assegnando ad ogni operatore logico
# una differente priorità: dovranno infatti essere prima eseguiti tutti gli AND
# da sinistra a destra, e poi tutti gli OR da sinistra a destra.
# (es) logic_op_priorities -> [7, 3]
def create_logic_ops(logic_op_positions):
  logic_op_priorities = []
  logic_ops = []
  for key, value in logic_op_positions.items():
    if value == "AND":
      logic_op_priorities.append(key)
  for key, value in logic_op_positions.items():
    if value == "OR":
      logic_op_priorities.append(key)

  # Vengono ora unite le posizioni, le priorità e i valori (AND/OR) 
  # di tutti gli operatori logici negli oggetti LogicOperator,
  # i quali vengono aggiunti alla lista logic_ops.
  # (es) logic_ops[0] -> {'pos': 3, 'ttype': 'OR', 'priority': 1, 'left': None, 'right': None, 'created_string': None}
  for key, value in logic_op_positions.items():
    op = LogicOperator()
    op.pos = key
    op.ttype = value
    op.priority = logic_op_priorities.index(key)
    logic_ops.append(op)
  return logic_ops

# Per ogni sottolista contenuta in where_list_2D viene creato un oggetto Block,
# e aggiunto alla lista blocks. Ogni blocco avrà come attributi l'id/posizione del blocco,
# il valore in sql (estratto da where_list_2D)
# ed il valore in mongodb (estratto da output_parenthesis).
# (es) blocks[0] -> {'id': 0, 'sql_value': ['status', '=', '"D"'], 'mongo_value': '{ status: { $eq: "D" }}', 'mapped_by': None}

def create_blocks(where_list_2D, output_parenthesis):
  blocks = []
  for i, item in enumerate(where_list_2D, start = 0):
    block = Block(i, item, output_parenthesis[i])
    blocks.append(block) 
  return blocks

# Ogni operatore logico in logic_ops viene mappato con la sottocondizione di sinistra (op.left)
# e con la sottocondizione di destra (op.right) in base alla sua posizione
# relativa nella condizione where.
# Le sottocondizioni possono essere dei blocchi (nel caso degli operatori con priorità
# di esecuzione maggiore) oppure il risultato di altri operatori eseguiti precedentemente.
# (es) logic_ops[0] -> {'pos': 7, 'ttype': 'AND', 'priority': 0, 'left': <__main__.Block object at 0x7f12d8b5ee80>, 'right': <__main__.Block object at 0x7f12d8b5ee48>, 'created_string': None}
def mapping(logic_ops, blocks):
  for op in logic_ops:
    rel_block_pos = op.pos//3
    left_block_id = rel_block_pos - 1
    right_block_id = rel_block_pos
    for block in blocks:
      if block.id == left_block_id and block.mapped_by == None:
        block.mapped_by = op
        op.left = block
      elif block.id == left_block_id and block.mapped_by != None:
        op.left = block.mapped_by
      elif block.id == right_block_id and block.mapped_by == None:
        block.mapped_by = op
        op.right = block
      elif block.id == right_block_id and block.mapped_by != None:
        op.right = block.mapped_by

# Viene eseguita la traduzione in mongoDB degli operatori, 
# partendo da quelli con priorità maggiore e memorizzando il risultato
# parziale nell'attributo created_string del blocco. Gli operatori successivi
# che hanno come mapping left o right quel blocco appena eseguito costruiranno
# in modo incrementale il risultato a partire dal valore di created_string.
# Il risultato finale della traduzione sarà quindi contenuto nell'attributo
# created_string dell'ultimo blocco.
def execute_ops (logic_ops):
  id_last_executed_op = None
  for op in logic_ops:
    if isinstance(op.left, Block) and isinstance(op.right, Block):
      left_value = op.left.mongo_value
      right_value = op.right.mongo_value
      id_last_executed_op = op.pos
    elif isinstance(op.left, LogicOperator) and isinstance(op.right, Block):
      left_value = find(logic_ops,id_last_executed_op).created_string
      right_value = op.right.mongo_value
      id_last_executed_op = op.pos
    elif isinstance(op.left, Block) and isinstance(op.right, LogicOperator):
      left_value = op.left.mongo_value
      right_value = find(logic_ops,id_last_executed_op).created_string
      id_last_executed_op = op.pos
    elif isinstance(op.left, LogicOperator) and isinstance(op.right, LogicOperator):
      left_value = find(logic_ops,id_last_executed_op).created_string
      right_value = find(logic_ops,id_last_executed_op).created_string
      id_last_executed_op = op.pos
    op.created_string  = "{$" + op.ttype.lower() + ": [" + str(left_value) + ", " +  str(right_value) + "]}"

def convert_condition_in_mongo(parsed):
  output_parenthesis = []
  selector = ""
  for item in parsed:
    if item == "=":
      selector = "$eq"
    elif item == "!=":
      selector = "$ne"
    elif item == ">":
      selector = "$gt"
    elif item == ">=":
      selector = "$gte"
    elif item == "<":
      selector = "$lt"
    elif item == "<=":
      selector = "$lte"
  output_inner = "{ "+ parsed[0] +": { "+selector+": " + parsed[2] + " }"
  output_parenthesis.append(output_inner + "}")
  return output_parenthesis

def convert_where_condition(token):
  parsed = token.value.split(" ")
  parsed.remove("WHERE")
  logic_op_positions = create_op_positions(parsed)
  if logic_op_positions:
    where_list_2D = create_subconditions_list(logic_op_positions, parsed)
    output_parenthesis = convert_subconditions_in_mongo(where_list_2D)
    logic_ops = create_logic_ops(logic_op_positions)
    blocks = create_blocks(where_list_2D, output_parenthesis)
   # Gli operatori logici vengono ordinati in base alla loro priorità di esecuzione.
    logic_ops.sort(key=lambda x: x.priority, reverse=False)
    mapping(logic_ops,blocks)
    execute_ops(logic_ops)
    output = logic_ops
  else:
    output = parsed
  return output

# Classe dell'operatore logico AND/OR. La pos indica la posizione all'interno della condizione where,
# (e quindi funge da id), il ttype indica il tipo AND/OR, la priorità l'ordine di esecuzione, left e right
# le sottocondizione di sinistra e di destra, mentre su created_string viene memorizzato il risultato 
# della sua traduzione in sintassi mongoDB.
class LogicOperator:
    def __init__(self, pos = None, ttype = None, priority = None, left = None, right = None, created_string = None):
      self.pos = pos
      self.ttype = ttype
      self.priority = priority
      self.left = left
      self.right = right
      self.created_string = created_string
    def __str__(self):
      return (str(self.__class__) + ": " + str(self.__dict__))


# La classe blocco indica una sottocondizione che precede o segue un operatore logico 
# nella condizione del where iniziale. Esso è quindi caratterizzato da un id (posizione del blocco), 
# da una traduzione in sql (sql_value) ed uno in mognodb (mongo_value).
# L'attributo mapped_by serve per mappare un operatore successivo con uno precedente 
# che ha già mappato quel blocco.
class Block:
  def __init__(self, id, sql_value = None, mongo_value = None, mapped_by = None):
    self.id = id
    self.sql_value = sql_value
    self.mongo_value = mongo_value
    self.mapped_by = mapped_by
  def __str__(self):
    return (str(self.__class__) + ": " + str(self.__dict__))

# Funzione per trovare un operatore logico in logic_ops tramite il suo id/pos
def find(list, id):
  result = None
  for item in list:
    if item.pos == id:
      result = item
  return result

"""
Created on Tue Feb 16 12:55:55 2021

@author: JOEL
"""

import sqlparse
from sqlparse.sql import IdentifierList, Identifier, Where, Function, Parenthesis, Values
import string

# Consultas que podemos rtealizar
#cadena1 = "INSERT INTO items (name,price) VALUES('Kellogs',2);"
def insert(tokens):
  tabla = ""
  #where_found = False
  columnas = []
  val_temporal = []
  valor = []

# 

#recorre todos los tokens para localizar el nombre de la tabla y encontrar los nombres de las columnas y los nombres de los valores,
# que se almacenan en las columnas y listas de valores respectivamente.
  for token in tokens:
    if isinstance(token, Function):
      tabla = token[0].value
      columnas = Encontrar_columna_nom(token)
    if isinstance(token,Values):
      val_temporal = Encontrar_columna_val(token)

# Aplicar formato a los elementos de los valores para eliminar la puntuación y los elementos vacíos
  valor = Valores_formato(val_temporal)

# calcula el nombre de la columna asociada para cada valor y crea la consulta mongodb correspondiente
  salida_parentesis = convertir_en_mongo(columnas, valor)
  
# Si se agrega más de una tupla, el resultado se incluye entre corchetes
  if (len(columnas) != len(valor)):
    salida_parentesis = "[" + salida_parentesis + "]"
  salida = "db." + tabla + ".insert( " + salida_parentesis + " )"
  return salida


def Encontrar_columna_nom(token):
  columnas = []
  for par in token:
    if isinstance(par,Parenthesis):
      for idlist in par:
        if isinstance(idlist,IdentifierList):
          for id in idlist:
            if isinstance(id,Identifier):
              columnas.append(id.value)
  return columnas

def Encontrar_columna_val(token):
  val_temporal = []
  for par in token:
    if isinstance(par,Parenthesis):
      for idlist in par:
        if isinstance(idlist,IdentifierList):
          for id in idlist:
            val_temporal.append(id.value)
  return val_temporal

def Valores_formato(val_temporal):
  valor = []
  new_s_p = string.punctuation.translate(str.maketrans('','','-'))
  for s in val_temporal:
      s = s.translate(str.maketrans('','',new_s_p))
      valor.append(s)
  valor = list(filter(None, valor))
  return valor

def convertir_en_mongo(columnas, valor):
  salida_parentesis = ""
  for i, value in enumerate(valor, start = 0):
    first_elem = ""
    last_elem = ""
    id_col = 0
    if i > 0 and len(columnas) > 0:
      id_col = i%(len(columnas))
    if id_col == 0:
      first_elem = "{"
    elif id_col == len(columnas)-1:
      last_elem = "}"
    if id_col < len(columnas):
      salida_parentesis = salida_parentesis + first_elem +columnas[id_col]  + ": '" + value + "'" + last_elem
    if value != valor[-1]:
      salida_parentesis = salida_parentesis + ", "
  return salida_parentesis


import tkinter as tk
from tkinter import*
import sqlparse#libreria de sql cup y lex

def Boton_Analizar(name):
    consulta_mql = " "
    consulta_sin = " "
    consulta = name
    formato = sqlparse.format(consulta, keyword_case="upper")
    analizado = sqlparse.parse(formato)  # analiza la primera cadena
    token_list = analizado[0]  # extraer la primera consulta (si hay más de una en la cadena)
    tokens = token_list.tokens  # extrae todas las palabras de la consulta y las clasifica según su tipo
    tipo_consulta = tokens[0].value
    if tipo_consulta == "SELECT":
            consulta_mql = select(tokens)
            consulta_sin = "CONSULTA DE TIPO SELECT"

    elif tipo_consulta == "INSERT":
        #print("consulta de tipo INSERT")
        consulta_mql = insert(tokens)
        consulta_sin = "CONSULTA DE TIPO INSERT"
    elif tipo_consulta == "DELETE":
        #print("consulta de tipo DELETE")
        consulta_mql = delete(tokens)
        consulta_sin = "CONSULTA DE TIPO DELETE"
    else:
        #consulta_mql = "CONSULTA DE SQL ERRADA ERROR EN LA LINEA 1 XD XD XD XD"
        consulta_sin = "CONSULTA DE SQL ERRADA se esperava un SELECT,FROM,INSERT"
    #print("LA CONSULTA REALIZADA ES: ", consulta_mql)
    print("LA CONSULTA REALIZADA ES: ",consulta_sin)
    print(consulta_mql)
    #print(consulta_sin)
    

#analiza texto de etrada sql
#print(consulta)
#name = 'SELECT*FROM alumno'

consulta = input()
lexema3 = analisis_lexico(consulta)
index = 0
print('\n\tANALISIS LEXICO')
for name, valor in lexema3:
  print('[',index,']',name,'\t  ==>  ',valor)
  index+=1
print('\n\tANALISIS SINTACTICO')
ERROR = Sintactico.analizar(lexema3)
if( ERROR == False):
    print('SINTAXIS CORRECTA')
    print('\n\tTRADUCTOR DE SQL A MQL')
Boton_Analizar(consulta)
#DELETE FROM nombre_tabla WHERE nombre_columna = valor
#INSERT INTO users(name, age) VALUES ('Marcela Sena', 29)
#SELECT * FROM CLIENTES


SELECT FROM CLIENTES

	ANALISIS LEXICO
[ 0 ] SELECT 	  ==>   SELECT
[ 1 ] FROM 	  ==>   FROM
[ 2 ] ID 	  ==>   CLIENTES

	ANALISIS SINTACTICO
ERROR DE SINTAXIS: en la linea 1 en " FROM "
LA CONSULTA REALIZADA ES:  CONSULTA DE TIPO SELECT
db..find({},{ CLIENTES: 1, _id: 0})
